In [51]:
import pandas as pd
import geopandas as gpd

from utils import *
import intake
import gcsfs

import calitp
from calitp.tables import tbl
from siuba import *

# Accessible Transit Metrics

### New Accessibilty Metric (Area)

The % of non-water area of California that is within 1/4 mi of a bus stop or 1 mi of a ferry/rail stop that is explicitly wheelchair accessible (and if in a station, that station has explicit pathways coding), and that has is served by a public-funded, open to the general public transit service with GTFS Schedule data that is served by a service that is explicitly wheelchair accessible

### New Accessibility Metric (Population)

 The % of Californians that are within 1/4 mi of a bus stop or 1 mi of a ferry/rail stop that is explicitly wheelchair accessible (and if in a station, that station has explicit pathways coding), and that has is served by a public-funded, open to the general public transit service with GTFS Schedule data that is served by a service that is explicitly wheelchair accessible

### Notes and Caveats:

* The vast majority of accessible services don't appear to provide accessibility data in GTFS
* Pathways appears to be empty in data warehouse for now...

In [8]:
total_pop_var = 'B01001_001E'

In [29]:
blockgrp_pop = get_census_ca_counties(total_pop_var, 'block%20group')
blockgrp_pop = blockgrp_pop.rename(columns={'B01001_001E': 'block_grp_pop',
                                           'block group': 'block_grp'})

In [30]:
blockgrp_pop.head(3)

,block_grp_pop,state,county,tract,block_grp
0,1388,6,47,1200,2
1,1531,6,47,1200,4
2,1405,6,47,1200,1


In [28]:
blockgrp_pop.columns

Index(['block_grp_pop', 'state', 'county', 'tract', 'block group'], dtype='object')

In [11]:
accessible_stops_trips = get_accessible_stops_and_trips()
## .25mi buffer on each accessible stop served by an accessible trip
accessible_stops_trips.geometry = accessible_stops_trips.buffer(400)

In [12]:
catalog = intake.open_catalog('./catalog.yml')

In [13]:
ca_block_geo = catalog.ca_block_groups.read()
ca_block_geo = ca_block_geo.to_crs('EPSG:4326')

In [14]:
stanford_shorelines = catalog.stanford_shorelines.read()

In [15]:
ca_shoreline = stanford_shorelines >> filter(_.STFIPS == '06')

In [16]:
ca_block_geo = ca_block_geo.clip(ca_shoreline)

In [17]:
ca_block_geo = ca_block_geo.to_crs('EPSG:6414')

In [18]:
## drop large block groups (not useful access data...)
## 4 sq km threshold
ca_block_geo = ca_block_geo[ca_block_geo.geometry.area < 4e+06]

In [31]:
geo_stops_joined = ca_block_geo.sjoin(accessible_stops_trips, how='inner', predicate='intersects')

In [32]:
geo_stops_joined = geo_stops_joined.drop_duplicates(subset=['GEOID'])

In [33]:
geo_stops_joined = geo_stops_joined.astype({'STATEFP': 'int64',
                                            'COUNTYFP': 'int64',
                                            'TRACTCE': 'int64',
                                            'BLKGRPCE': 'int64'})
geo_stops_joined = geo_stops_joined.rename(columns={'STATEFP': 'state',
                                                   'COUNTYFP': 'county',
                                                   'TRACTCE': 'tract',
                                                   'BLKGRPCE': 'block_grp'})
geo_stops_joined.head(3)

,state,county,tract,block_grp,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,INTPTLON,geometry,index_right,calitp_itp_id,calitp_url_number,stop_id,wheelchair_accessible,stop_lat,stop_lon,wheelchair_boarding
16760,6,73,10009,1,060730100091,Block Group 1,G5030,S,1759046,528681,...,-117.0559349,"MULTIPOLYGON (((275529.078 -603346.514, 275528...",2198,278,0,60313,1,32.552096,-117.044117,1
18465,6,73,10009,3,060730100093,Block Group 3,G5030,S,707892,0,...,-117.0543357,"POLYGON ((276297.187 -601745.532, 276297.401 -...",2198,278,0,60313,1,32.552096,-117.044117,1
18392,6,73,10013,1,060730100131,Block Group 1,G5030,S,491924,0,...,-117.0447331,"POLYGON ((277107.189 -601286.509, 277150.566 -...",2198,278,0,60313,1,32.552096,-117.044117,1


In [35]:
stops_pop_joined = geo_stops_joined >> inner_join(_, blockgrp_pop, on=['state', 'county', 'tract', 'block_grp'])

In [36]:
stops_pop_joined['block_grp_pop'].sum()

2462477

In [37]:
blockgrp_pop['block_grp_pop'].sum()

39283497

#### Current Percentage of CA population in block groups within .25mi of accessible transit

In [39]:
(stops_pop_joined['block_grp_pop'].sum() / blockgrp_pop['block_grp_pop'].sum()).round(3) * 100

6.3

In [42]:
geo_stops_joined.geometry.area.sum() ## sq meters

993807966.9908118

In [43]:
ca_block_geo.geometry.area.sum()

14393648284.97847

#### Current Percentage of CA land area in block groups within .25mi of accessible transit

In [45]:
(geo_stops_joined.geometry.area.sum() / ca_block_geo.geometry.area.sum()).round(3) * 100

6.9

### Map

In [50]:
simple_map(geo_stops_joined, 'calitp_itp_id')

Map(center=[32.54594464811505, -117.05710800819634], controls=(ZoomControl(options=['position', 'zoom_in_text'…

#### Static Map (block groups near accessible transit, Bay Area)

![bay area accessiblity](accessible.png)

# General Transit Metrics

### New General Metric (by area):

The % of non-water area of Californian that is within 1/4 mi of a bus stop or 1 mi of a ferry/rail stop that has is served by a public-funded, open to the general public transit service with GTFS Schedule data


### New General Metric (by population):

The % of Californians that live within 1/4 mi of a bus stop or 1 mi of a ferry/rail stop that has is served by a public-funded, open to the general public transit service with GTFS Schedule data
